In [1]:
import pickle
import pandas as pd
from geopy.distance import geodesic

with open("..\\models\\modelo_lightgbm.pkl", "rb") as model_file:
    model = pickle.load(model_file)

with open("..\\models\\scaler.pkl", "rb") as scaler_file:
    scaler = pickle.load(scaler_file)

with open("..\\models\\encoder.pkl", "rb") as encoder_file:
    encoder = pickle.load(encoder_file)

print("Modelo, scaler e encoder carregados!")

dados_teste = pd.DataFrame([{
    "latitude": 40.75362,
    "longitude": -73.98377,
    "bairro_group": "Manhattan",
    "room_type": "Entire home/apt",
    "minimo_noites": 1,
    "numero_de_reviews": 45,
    "disponibilidade_365": 355,
    "calculado_host_listings_count": 2
}])

# Precisamos calcular a distância até o centro e adicionar ao dataframe
centro = (40.7580, -73.9855)
dados_teste["distancia_centro"] = geodesic((dados_teste["latitude"].values[0], dados_teste["longitude"].values[0]), centro).km

# Além disso precisamos aplicar o scaler também e o one-hot-encoding
num_features = ["latitude", "longitude", "distancia_centro", "minimo_noites"]
dados_teste[num_features] = scaler.transform(dados_teste[num_features])  # Usar o scaler treinado
dados_categ_encoded = encoder.transform(dados_teste[["bairro_group", "room_type"]])
dados_teste_final = pd.concat(
    [
        dados_teste[num_features].reset_index(drop=True),
        pd.DataFrame(dados_categ_encoded, columns=encoder.get_feature_names_out()),
    ],
    axis=1,
)

#Com o dataframe pronto basta passar para o modelo prever
preco_previsto = model.predict(dados_teste_final)

print(f"Preço previsto: ${preco_previsto[0]:.2f}")

Modelo, scaler e encoder carregados!
Preço previsto: $154.80
